## Atividade Global 2
**Alunos**: 
- Gabriel Tiso Vinhas de Brito - 236 - GES
- Diego de Jesus Rodrigues - 233 - GES

In [21]:
import pandas as pd

#### Tratamento dos dados

In [22]:
wholesale_dataset = pd.read_csv("./wholesale.csv")
wholesale_dataset.head()

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicatessen
0,Retail,Other,12669,9656,7561,214,2674,1338
1,Retail,Other,7057,9810,9568,1762,3293,1776
2,Retail,Other,6353,8808,7684,2405,3516,7844
3,HoReCa,Other,13265,1196,4221,6404,507,1788
4,Retail,Other,22615,5410,7198,3915,1777,5185


In [23]:
channelMapping = { "HoReCa": 0, "Retail": 1 }
regionMapping = { "Lisbon": 0, "Oporto": 1, "Other": 2 }

# Realizando o mapeamento dos valores de strings para inteiros
wholesale_dataset["Channel"] = wholesale_dataset["Channel"].replace(channelMapping)
wholesale_dataset["Region"] = wholesale_dataset["Region"].replace(regionMapping)

# Reordenando as colunas de acordo com o enunciado
new_index = ["Region", "Fresh", "Milk", "Grocery", "Frozen", "Detergents_Paper", "Delicatessen", "Channel"]
wholesale_dataset = wholesale_dataset.reindex(columns=new_index)

# Mostrando os resultados
wholesale_dataset.head()

/var/folders/rq/t6yr91rd3ns19ph7zyfvtm_40000gn/T/ipykernel_82863/3201846847.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  wholesale_dataset["Channel"] = wholesale_dataset["Channel"].replace(channelMapping)
/var/folders/rq/t6yr91rd3ns19ph7zyfvtm_40000gn/T/ipykernel_82863/3201846847.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  wholesale_dataset["Region"] = wholesale_dataset["Region"].replace(regionMapping)


,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicatessen,Channel
0,2,12669,9656,7561,214,2674,1338,1
1,2,7057,9810,9568,1762,3293,1776,1
2,2,6353,8808,7684,2405,3516,7844,1
3,2,13265,1196,4221,6404,507,1788,0
4,2,22615,5410,7198,3915,1777,5185,1


#### Modelo de Classificação

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
RANDOM_STATE = 42

# De acordo com o texto, podemos observar que o que queremos classificar é o canal pelo qual a venda dos 
# produtos é realizada. O campo "Channel" do dataset deve ser portanto nosso alvo.
x = wholesale_dataset.drop("Channel", axis=1)
y = wholesale_dataset["Channel"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=RANDOM_STATE)

In [26]:
from sklearn.neural_network import MLPClassifier

# Instanciar e treinar o modelo com os dados de teste
classifier = MLPClassifier(random_state=RANDOM_STATE)
classifier.fit(x_train, y_train)

# Fazer a predição
y_pred = classifier.predict(x_test)

In [27]:
# Gerar o relatório
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.72      0.83        65
           1       0.55      0.96      0.70        23

    accuracy                           0.78        88
   macro avg       0.76      0.84      0.77        88
weighted avg       0.87      0.78      0.80        88



#### Entradas escolhidas pelo usuário

In [28]:
# Para predição de entradas de usuário, basta requisitarmnos as informações necessárias dos gastos e regiões.
# Em seguida, basta unir essas estruturas em um dataframe e usar a função `predict` para obter os resultados
valid_regions = ["Lisbon", "Oporto", "Other"]
spending_labels = ["Fresh", "Milk", "Grocery", "Frozen", "Detergents_Paper", "Delicatessen"]
user_data = []

op = 1
while op == 1:
    labels = {}

    # Região é um pouco diferente das outras labels, por isso pegamos ela aqui
    region = input("Região onde a venda foi realizada: ")
    if region not in valid_regions:
        print(f"Região não suportada, apenas {valid_regions} são regiões válidas")
        continue
    labels["Region"] = region

    # O resto das labels
    for sl in spending_labels:
        value = float(input(f"Gasto em u.m com produtos da categoria `{sl}`:"))
        labels[sl] = value

    user_data.append(labels)
    op = int(input("Digite 1 para inserir mais dados, e 0 para sair: "))

# Transformar os dados em um DataFrame
df = pd.DataFrame(user_data)
df

,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicatessen
0,Other,3000.0,10000.0,23000.0,10000.0,3400.0,500.0


In [30]:
regionMapping = { "Lisbon": 0, "Oporto": 1, "Other": 2 }
df["Region"] = df["Region"].replace(regionMapping)

# Nesse momento, após aplicar as transformações desejadas no dataset, basta chamar o método `predict`
user_data_prediction = classifier.predict(df)
prediction_df = pd.DataFrame(user_data_prediction, columns=["Channel"])
prediction_df["Channel"] = prediction_df["Channel"].replace({0: "HoReCa", 1: "Retail"})

prediction_df.head()

,Channel
0,Retail
